# Send

By default, `Node`  and `Edge` are defined ahead of time and operate on the same shared state. A common example of this is with map-reduce design pattern. In this design the first node may generate the list of objects. and you want to apply some other node to all those objects. The number of object may be unkonwn and the input State to the downstream Node should be different (one for each generated object).

To do this We have `Send` that return object from conditional edges. `Send` accept two aguments `name_of_node` and `state_to_pass_that_node`.

* Example ⬇️

In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from typing import Annotated,List
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

list_of_topic = ["Moon", "Sun", "Earth"]

class State(TypedDict):
    list_of_topic: List
    Result:Annotated[list,add_messages]

builder = StateGraph(State)


def first_node(state: State):
    print("<------------------------- FIRST_NODE --------------------------->")
    return {"Result" : state["list_of_topic"]}

def get_topic_info(state: dict):
    print("<------------------------ GETTING_INFO --------------------------->")
    match state["topic"]:
        case "Moon":
            return {"Result" : "Moon the bright and white"}
        case "Sun":
            return {"Result" : "Sun is bright and yellow"}
        case "Earth":
            return {"Result" : "Life line of human"}

def continue_to_info(state: State):
    print("<------------------------- CALL_TO_GET_INFO ---------------------->")
    return [Send("get_topic_info", {"topic": t}) for t in state['Result']]

builder.add_node(first_node)
builder.add_node(continue_to_info)
builder.add_node(get_topic_info)

builder.add_edge(START,"first_node")
builder.add_conditional_edges("first_node", continue_to_info)
builder.add_edge("first_node", END)
builder.add_edge("get_topic_info", END)  # Add this line to allow get_topic_info to terminate

graph = builder.compile()

print(graph.invoke({"list_of_topic": list_of_topic}))

<------------------------- FIRST_NODE --------------------------->
<------------------------- CALL_TO_GET_INFO ---------------------->
<------------------------ GETTING_INFO --------------------------->
<------------------------ GETTING_INFO --------------------------->
<------------------------ GETTING_INFO --------------------------->
{'list_of_topic': ['Moon', 'Sun', 'Earth'], 'Result': [HumanMessage(content='Moon', additional_kwargs={}, response_metadata={}, id='753c00cf-3756-40ce-b1d4-388375fc20cd'), HumanMessage(content='Sun', additional_kwargs={}, response_metadata={}, id='82df4981-3d1c-4093-afff-7da496347929'), HumanMessage(content='Earth', additional_kwargs={}, response_metadata={}, id='e2d6c8d8-90eb-4a92-907c-b90dc8861c78')]}
